In [251]:
import pandas as pd
import numpy as np
from scipy.stats import mode
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

In [252]:
d=pd.read_csv('C:/Users/Logeshwaran R/Documents/FoDs Assignments/Kaggle/train.csv')

In [253]:
a=d.copy()


In [254]:
a['charges_2 (%)']=a['charges_2 (%)'].fillna(a.groupby(['Product_Category','Grade'])['charges_2 (%)'].transform('mean'))

In [255]:
a['charges_1']=a['charges_1'].fillna(a.groupby(['Product_Category','Grade'])['charges_1'].transform('mean'))

In [256]:
a['Discount_avail']=a['Discount_avail'].fillna(0)
a['Minimum_price'] = a['Minimum_price'].fillna(a['Minimum_price'].mean())
a['Maximum_price'] = a['Maximum_price'].fillna(a['Maximum_price'].mean())
a['Selling_Price'] = a['Selling_Price'].fillna(a['Selling_Price'].mean())

In [257]:
common= a.pivot_table(values='Stall_no', columns=['Product_Category'],aggfunc=(lambda x:mode(x)[0]) )
common

Product_Category,Child_care,Cosmetics,Educational,Fashion,Home_decor,Hospitality,Organic,Pet_care,Repair,Technology
Stall_no,50.0,24.0,9.0,12.0,48.0,32.0,11.0,37.0,20.0,2.0


In [258]:
temp= a['Stall_no'].isnull()
a.loc[temp,'Stall_no'] = a.loc[temp,'Product_Category'].apply(lambda x: common[x])

In [259]:
a.isnull().sum()

Product_id            0
Stall_no              0
instock_date          0
Market_Category       0
Customer_name       159
Loyalty_customer      0
Product_Category      0
Grade                 0
Demand                0
Discount_avail        0
charges_1             0
charges_2 (%)         0
Minimum_price         0
Maximum_price         0
Selling_Price         0
dtype: int64

In [260]:
data_train=a.copy()

In [261]:
model=IsolationForest(n_estimators=50,max_samples='auto',contamination=float(0.1),random_state=1,max_features=1.0)

In [262]:
data_train.shape

(4368, 15)

In [263]:
labelencoder=LabelEncoder()
data_train[['Loyalty_customer']] = labelencoder.fit_transform(data_train[['Loyalty_customer']])
data_train[['Product_Category']]=labelencoder.fit_transform(data_train[['Product_Category']])
data_train['charges_2 (%)']=((data_train['Minimum_price']+data_train['Maximum_price'])*data_train['charges_2 (%)'])/200
data_train

c:\users\logeshwaran r\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,36:12.0,2,Lillyann,1,3,1,68,0.0,376.0,423.280,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,19:13.0,24,Klynn,1,3,0,51,0.0,397.0,1070.820,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,25:22.0,447,Ridge,1,0,0,10,0.0,250.0,587.745,5752.0,7309.0,6785.701362
3,JWSEBUKYQPMBZ3RK,7.0,58:53.0,63,Dustyn,1,8,1,35,1.0,211.0,233.820,2430.0,9261.0,906.553935
4,SHOE6QGAYFFMBWG6,4.0,29:06.0,23,Koby,1,2,1,67,0.0,104.0,713.775,3596.0,5921.0,4820.943600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4363,SWSEBHCP5YSCUU2S,13.0,09:55.0,358,Adelaide,0,8,3,50,0.0,266.0,260.685,3205.0,14174.0,8900.778096
4364,MUGE8DAUYGPWG9AE,17.0,21:09.0,358,Roxanna,1,2,3,7,0.0,144.0,60.390,487.0,855.0,4538.353940
4365,SHOEF9G8Z4ZXF6WE,21.0,09:26.0,55,Theodore,1,0,0,35,0.0,269.0,212.100,3564.0,4920.0,4501.837200
4366,PBXEB9ZQ7HHCFXYG,36.0,29:21.0,358,Galen,0,8,3,29,0.0,283.0,298.755,5303.0,14614.0,10218.878780


In [264]:
data_train.isnull().sum()

Product_id            0
Stall_no              0
instock_date          0
Market_Category       0
Customer_name       159
Loyalty_customer      0
Product_Category      0
Grade                 0
Demand                0
Discount_avail        0
charges_1             0
charges_2 (%)         0
Minimum_price         0
Maximum_price         0
Selling_Price         0
dtype: int64

In [265]:
X=data_train.drop(['Product_id','instock_date','Customer_name','Selling_Price'],axis=1)
Y=data_train[['Selling_Price']].values



X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.33,random_state=0)
reg2 = LinearRegression()
reg2.fit(X_train,y_train)
y_pred = reg2.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = round(np.sqrt(mse),3)

In [266]:
X

,Stall_no,Market_Category,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,37.0,2,1,3,1,68,0.0,376.0,423.280,2983.0,4713.0
1,38.0,24,1,3,0,51,0.0,397.0,1070.820,7495.0,10352.0
2,9.0,447,1,0,0,10,0.0,250.0,587.745,5752.0,7309.0
3,7.0,63,1,8,1,35,1.0,211.0,233.820,2430.0,9261.0
4,4.0,23,1,2,1,67,0.0,104.0,713.775,3596.0,5921.0
...,...,...,...,...,...,...,...,...,...,...,...
4363,13.0,358,0,8,3,50,0.0,266.0,260.685,3205.0,14174.0
4364,17.0,358,1,2,3,7,0.0,144.0,60.390,487.0,855.0
4365,21.0,55,1,0,0,35,0.0,269.0,212.100,3564.0,4920.0
4366,36.0,358,0,8,3,29,0.0,283.0,298.755,5303.0,14614.0


In [267]:
from sklearn.metrics import r2_score
print("R2-score: %.2f" % r2_score(y_test,y_pred) )

R2-score: 0.89


In [268]:
rmse

983.156

In [269]:
test=pd.read_csv('C:/Users/Logeshwaran R/Documents/FoDs Assignments/Kaggle/test_upload.csv')
sample=pd.read_csv('C:/Users/Logeshwaran R/Documents/FoDs Assignments/Kaggle/sample_submission.csv')

In [270]:
test['charges_1']=test['charges_1'].fillna(test.groupby(['Product_Category','Grade'])['charges_1'].transform('mean'))
test['charges_2 (%)']=test['charges_2 (%)'].fillna(test.groupby(['Product_Category','Grade'])['charges_2 (%)'].transform('mean'))
test['Discount_avail']=test['Discount_avail'].fillna(0)
test['Minimum_price'] = test['Minimum_price'].fillna(test['Minimum_price'].mean())
test['Maximum_price'] = test['Maximum_price'].fillna(test['Maximum_price'].mean())


In [271]:
common= test.pivot_table(values='Stall_no', columns=['Product_Category'],aggfunc=(lambda x:mode(x)[0]) )
common

Product_Category,Child_care,Cosmetics,Educational,Fashion,Home_decor,Hospitality,Organic,Pet_care,Repair,Technology
Stall_no,30.0,25.0,18.0,1.0,32.0,20.0,15.0,26.0,34.0,23.0


In [272]:
temp= test['Stall_no'].isnull()
test.loc[temp,'Stall_no'] = test.loc[temp,'Product_Category'].apply(lambda x: common[x])

In [273]:
data=test.copy()
labelencoder=LabelEncoder()
data[['Loyalty_customer']] = labelencoder.fit_transform(data[['Loyalty_customer']])
data[['Product_Category']]=labelencoder.fit_transform(data[['Product_Category']])
data['charges_2 (%)']=((data['Minimum_price']+data['Maximum_price'])*data['charges_2 (%)'])/200
data

c:\users\logeshwaran r\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price
0,BLAE6DPHRB25VEHK,10.0,08:04.0,358,Addalyn,1,1,3,64,0.0,270.000000,1865.920,8335.0,13617.0
1,RTRDYYTZVSFZQXWF,49.0,38:43.0,115,Crimson,0,5,0,8,0.0,248.000000,216.420,1429.0,2178.0
2,PBXEAP3SGKSH89XC,43.0,37:01.0,220,Eesha,1,4,0,49,0.0,233.000000,642.880,3755.0,5429.0
3,TOPDQAUYZKH2GDTM,5.0,48:23.0,14,Annistyn,1,7,0,51,0.0,155.000000,268.310,3339.0,4327.0
4,PCSE76X4QXVW9KG9,49.0,46:52.0,24,Jameria,1,4,0,56,1.0,275.000000,1305.685,6442.0,8919.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,INKECA5VFTCGF77E,37.0,48:45.0,363,Hoyt,0,6,3,27,0.0,132.000000,63.455,457.0,1356.0
1996,ACCEHHR5ESZAXDPA,1.0,06:35.0,451,Skai,1,5,0,15,0.0,326.000000,855.470,5454.0,6767.0
1997,SHOEBGDMKUF33QYC,36.0,25:02.0,65,Kaliah,0,5,2,15,0.0,218.000000,127.080,689.0,2488.0
1998,DUPEGNY4RYYXGJH6,29.0,19:42.0,63,Livia,0,1,1,6,1.0,244.636364,1023.075,3882.0,9759.0


In [274]:
data.isnull().sum()

Product_id           0
Stall_no             0
instock_date         0
Market_Category      0
Customer_name       52
Loyalty_customer     0
Product_Category     0
Grade                0
Demand               0
Discount_avail       0
charges_1            0
charges_2 (%)        0
Minimum_price        0
Maximum_price        0
dtype: int64

In [275]:
X_train=data_train.drop(['Product_id','instock_date','Customer_name','Selling_Price'],axis=1)
Y_train=data_train[['Selling_Price']].values


X_test=data.drop(['Product_id','instock_date','Customer_name'],axis=1)


reg2 = LinearRegression()
reg2.fit(X_train,Y_train)
data['Selling_Price'] = reg2.predict(X_test)

In [276]:
sample_sub=data[['Product_id','Selling_Price']]

## 


In [277]:
sample_sub

,Product_id,Selling_Price
0,BLAE6DPHRB25VEHK,10467.327632
1,RTRDYYTZVSFZQXWF,2371.267597
2,PBXEAP3SGKSH89XC,4754.226796
3,TOPDQAUYZKH2GDTM,4043.620346
4,PCSE76X4QXVW9KG9,3113.745507
...,...,...
1995,INKECA5VFTCGF77E,1569.594724
1996,ACCEHHR5ESZAXDPA,6223.857112
1997,SHOEBGDMKUF33QYC,2208.621476
1998,DUPEGNY4RYYXGJH6,2413.006117


In [278]:
sample


,Product_id,Selling_Price
0,BLAE6DPHRB25VEHK,0
1,RTRDYYTZVSFZQXWF,0
2,PBXEAP3SGKSH89XC,0
3,TOPDQAUYZKH2GDTM,0
4,PCSE76X4QXVW9KG9,0
...,...,...
1982,INKECA5VFTCGF77E,0
1983,ACCEHHR5ESZAXDPA,0
1984,SHOEBGDMKUF33QYC,0
1985,DUPEGNY4RYYXGJH6,0


In [279]:
l=sample_sub.merge(sample,left_on=sample_sub['Product_id'],right_on=sample['Product_id'])

In [280]:
l

,key_0,Product_id_x,Selling_Price_x,Product_id_y,Selling_Price_y
0,BLAE6DPHRB25VEHK,BLAE6DPHRB25VEHK,10467.327632,BLAE6DPHRB25VEHK,0
1,RTRDYYTZVSFZQXWF,RTRDYYTZVSFZQXWF,2371.267597,RTRDYYTZVSFZQXWF,0
2,PBXEAP3SGKSH89XC,PBXEAP3SGKSH89XC,4754.226796,PBXEAP3SGKSH89XC,0
3,TOPDQAUYZKH2GDTM,TOPDQAUYZKH2GDTM,4043.620346,TOPDQAUYZKH2GDTM,0
4,PCSE76X4QXVW9KG9,PCSE76X4QXVW9KG9,3113.745507,PCSE76X4QXVW9KG9,0
...,...,...,...,...,...
1982,INKECA5VFTCGF77E,INKECA5VFTCGF77E,1569.594724,INKECA5VFTCGF77E,0
1983,ACCEHHR5ESZAXDPA,ACCEHHR5ESZAXDPA,6223.857112,ACCEHHR5ESZAXDPA,0
1984,SHOEBGDMKUF33QYC,SHOEBGDMKUF33QYC,2208.621476,SHOEBGDMKUF33QYC,0
1985,DUPEGNY4RYYXGJH6,DUPEGNY4RYYXGJH6,2413.006117,DUPEGNY4RYYXGJH6,0


In [281]:
final=l[['Product_id_x','Selling_Price_x']]

In [282]:
final.columns=sample.columns

In [283]:
final.reset_index(drop=True, inplace=True)

In [284]:
final

,Product_id,Selling_Price
0,BLAE6DPHRB25VEHK,10467.327632
1,RTRDYYTZVSFZQXWF,2371.267597
2,PBXEAP3SGKSH89XC,4754.226796
3,TOPDQAUYZKH2GDTM,4043.620346
4,PCSE76X4QXVW9KG9,3113.745507
...,...,...
1982,INKECA5VFTCGF77E,1569.594724
1983,ACCEHHR5ESZAXDPA,6223.857112
1984,SHOEBGDMKUF33QYC,2208.621476
1985,DUPEGNY4RYYXGJH6,2413.006117


In [285]:
for i in range(0,len(final)):
    
    if(final['Selling_Price'][i]<0):
        final['Selling_Price'][i]=0

<ipython-input-285-28d1846c39da>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Selling_Price'][i]=0
C:\Users\Logeshwaran R\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3418: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [286]:
final

,Product_id,Selling_Price
0,BLAE6DPHRB25VEHK,10467.327632
1,RTRDYYTZVSFZQXWF,2371.267597
2,PBXEAP3SGKSH89XC,4754.226796
3,TOPDQAUYZKH2GDTM,4043.620346
4,PCSE76X4QXVW9KG9,3113.745507
...,...,...
1982,INKECA5VFTCGF77E,1569.594724
1983,ACCEHHR5ESZAXDPA,6223.857112
1984,SHOEBGDMKUF33QYC,2208.621476
1985,DUPEGNY4RYYXGJH6,2413.006117


In [287]:
final.to_csv('submission_out5.csv',index=False)

In [288]:
sample

,Product_id,Selling_Price
0,BLAE6DPHRB25VEHK,0
1,RTRDYYTZVSFZQXWF,0
2,PBXEAP3SGKSH89XC,0
3,TOPDQAUYZKH2GDTM,0
4,PCSE76X4QXVW9KG9,0
...,...,...
1982,INKECA5VFTCGF77E,0
1983,ACCEHHR5ESZAXDPA,0
1984,SHOEBGDMKUF33QYC,0
1985,DUPEGNY4RYYXGJH6,0


In [289]:
data_train[data_train['Selling_Price']<0].size

150

In [290]:
final[final['Selling_Price']<0].size

0